<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-01 13:44:30
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.40 C
-------------------
Today PnL: 1.00 L (0.72%)
Current PnL: -21.80 L (-14.47%)
CY Booked + Current PnL: -10.36 L (-6.88%)
-------------------
Total profit:  1.18 L
Total loss:  -22.98 L
-------------------
Total Booked + Current PnL: 16.62 L (13.43%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.18%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 89.12 L (63.75%)
Deployed:  1.24 C
Current:  1.40 C
CAGR/XIRR %: 7.48%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-43.66,62.0,H-MC,12.52,113070.0,12566.0,16542.0,-0.25,12.50,14.63,28.96,79.0,0.76,0.81,50.72,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.38,36.0,M-SC,2.45,83309.0,-17468.0,17562.0,0.16,-17.33,21.08,0.09,245.0,-0.99,0.60,9.45,OX40N,NTT,DURABLES
42,ITC,452.00,-38.77,47.0,H-LC,2.04,198021.0,-2117.0,23010.0,0.85,-1.06,11.62,10.44,4.0,-0.09,1.42,4.45,X40,NTT,FMCG
14,BLUESTARCO,2080.00,7.18,41.0,H-MC,3.36,184950.0,20280.0,23045.0,-2.43,12.32,12.46,26.31,89.0,0.88,1.33,21.28,X40N,NTT,AC
57,RELIANCE,1533.00,-14.79,40.0,H-LC,3.70,212956.0,2590.0,26194.0,0.08,1.23,12.30,13.68,37.0,0.10,1.53,17.96,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.00,68.86,78.0,H-SC,1.76,155907.0,-23164.0,88898.0,15.02,-12.94,57.02,36.71,135.0,-0.26,1.12,54.69,XR,NTT,BANKS
26,GREENPANEL,537.00,235.27,59.0,M-SC,1.39,149743.0,-33335.0,112307.0,11.18,-18.21,75.00,43.14,230.0,-0.30,1.08,40.85,XY24,NTT,MISC
64,SHALBY,327.00,1244.50,69.0,M-SC,14.28,182779.0,1636.0,43830.0,6.73,0.90,23.98,25.10,235.0,0.04,1.31,46.48,XY24,NTT,HEALTHCARE
71,TANLA,1963.11,-21.56,51.0,H-SC,17.93,210666.0,-60095.0,390048.0,4.83,-22.19,185.15,121.86,133.0,-0.15,1.51,64.80,AR,ATH,IT
73,TATAMOTORS,1065.00,-18.21,58.0,M-LC,2.26,258638.0,-17885.0,127948.0,4.75,-6.47,49.47,39.81,54.0,-0.14,1.86,23.95,XY24,NTT,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-47.86,23.0,M-SC,1.19,88072.0,-3426.0,82180.0,-4.73,-3.74,93.31,86.07,223.0,-0.04,0.63,27.90,XR,NTT,DURABLES
55,RAJOOENG,143.10,-46.04,19.0,H-SC,14.29,81760.0,-20740.0,61336.0,-4.40,-20.23,75.02,39.61,114.0,-0.34,0.59,0.00,AR,ATH,MISC
3,ALKYLAMINE,4546.37,-12.24,28.0,H-SC,10.68,81752.0,-19211.0,113742.0,-3.50,-19.03,139.13,93.63,148.0,-0.17,0.59,21.77,SR,ATH,CHEMICALS
81,VAIBHAVGBL,521.00,47.11,43.0,H-SC,5.81,127605.0,-55170.0,166244.0,-2.68,-30.18,130.28,60.77,125.0,-0.33,0.92,14.29,XR,NTT,JEWELLERY
14,BLUESTARCO,2080.00,7.18,41.0,H-MC,3.36,184950.0,20280.0,23045.0,-2.43,12.32,12.46,26.31,89.0,0.88,1.33,21.28,X40N,NTT,AC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1244.50,69.0,M-SC,14.28,182779.0,1636.0,43830.0,6.73,0.90,23.98,25.10,235.0,0.04,1.31,46.48,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.0,-186.36,80.0,M-SC,8.28,149931.0,711.0,143469.0,3.56,0.48,95.69,96.62,208.0,0.00,1.08,62.26,XY25,NTT,FINANCE
35,ICICIPRULI,790.0,-21.80,44.0,H-MC,2.21,134583.0,-1205.0,43955.0,1.71,-0.89,32.66,31.48,107.0,-0.03,0.97,11.12,X40,ATH,INSURANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,105.37,64.0,H-SC,11.45,129735.0,-12744.0,32265.0,3.67,-8.94,24.87,13.70,163.0,-0.39,0.93,53.12,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.61,54.0,H-SC,1.56,223146.0,-46521.0,82854.0,0.55,-17.25,37.13,13.47,138.0,-0.56,1.60,12.86,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,93.38,36.0,M-SC,2.45,83309.0,-17468.0,17562.0,0.16,-17.33,21.08,0.09,245.0,-0.99,0.60,9.45,OX40N,NTT,DURABLES
66,SIS,528.0,2102.59,44.0,H-SC,4.04,88880.0,-22152.0,45755.0,0.97,-19.95,51.48,21.26,156.0,-0.48,0.64,20.09,OX40N,NTT,MISC
18,CERA,9475.0,-23.15,35.0,H-SC,1.62,140300.0,-35603.0,77628.0,-0.95,-20.24,55.33,23.89,149.0,-0.46,1.01,21.17,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1244.50,69.0,M-SC,14.28,182779.0,1636.0,43830.0,6.73,0.90,23.98,25.10,235.0,0.04,1.31,46.48,XY24,NTT,HEALTHCARE
60,SAIL,228.00,46.75,59.0,M-MC,11.49,231557.0,6595.0,161048.0,-0.01,2.93,69.55,74.53,192.0,0.04,1.66,36.13,XY24,BTT,STEEL
32,HINDZINC,730.22,28.52,65.0,M-LC,3.86,209452.0,4376.0,108203.0,-0.20,2.13,51.66,54.89,52.0,0.04,1.50,26.74,X5K,ATH,METALS
14,BLUESTARCO,2080.00,7.18,41.0,H-MC,3.36,184950.0,20280.0,23045.0,-2.43,12.32,12.46,26.31,89.0,0.88,1.33,21.28,X40N,NTT,AC
84,VOLTAS,1530.00,-3.46,42.0,H-MC,2.83,202575.0,10833.0,26922.0,0.77,5.65,13.29,19.69,99.0,0.40,1.46,12.83,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-43.66,62.0,H-MC,12.52,113070.0,12566.0,16542.0,-0.25,12.50,14.63,28.96,79.0,0.76,0.81,50.72,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,67.0,H-MC,9.46,191896.0,12088.0,100323.0,1.10,6.72,52.28,62.52,88.0,0.12,1.38,43.10,XR,NTT,BANKS
32,HINDZINC,730.22,28.52,65.0,M-LC,3.86,209452.0,4376.0,108203.0,-0.20,2.13,51.66,54.89,52.0,0.04,1.50,26.74,X5K,ATH,METALS
37,INDIAMART,4850.92,-54.83,29.0,H-SC,10.00,126193.0,2857.0,130900.0,0.60,2.32,103.73,108.45,119.0,0.02,0.91,25.19,AR,ATH,MISC
86,WIPRO,420.00,-15.61,39.0,M-LC,5.80,148602.0,-2343.0,111793.0,0.13,-1.55,75.23,72.51,53.0,-0.02,1.07,4.38,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.81,27.0,H-LC,5.82,204776.0,-46992.0,100197.0,-0.98,-18.66,48.93,21.13,27.0,-0.47,1.47,10.15,X40,ATH,PAINTS
74,TCS,4476.75,-31.58,30.0,H-LC,12.41,272732.0,-73224.0,148093.0,0.45,-21.17,54.30,21.64,1.0,-0.49,1.96,0.45,X40,ATH,IT
83,VBL,671.64,-21.40,32.0,H-LC,9.26,282889.0,-32953.0,145603.0,-0.07,-10.43,51.47,35.67,5.0,-0.23,2.03,1.98,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.95,32.0,H-LC,10.26,217272.0,-24652.0,87930.0,-0.09,-10.19,40.47,26.15,8.0,-0.28,1.56,2.67,X40,ATH,IT
44,JIOFIN,387.00,-5.85,34.0,H-LC,12.88,255850.0,-11515.0,76960.0,1.47,-4.31,30.08,24.48,48.0,-0.15,1.84,48.30,XY24,BTT,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-7.51,41.0,H-LC,1.58,229005.0,-17146.0,96365.0,-1.34,-6.97,42.08,32.18,16.0,-0.18,1.65,26.57,X200,ATH,IT
65,SIEMENS,4671.5,-3.25,43.0,H-LC,1.98,155605.0,-30490.0,77974.0,-0.55,-16.38,50.11,25.51,15.0,-0.39,1.12,14.67,AR,ATH,ELECTRICAL
42,ITC,452.0,-38.77,47.0,H-LC,2.04,198021.0,-2117.0,23010.0,0.85,-1.06,11.62,10.44,4.0,-0.09,1.42,4.45,X40,NTT,FMCG
57,RELIANCE,1533.0,-14.79,40.0,H-LC,3.70,212956.0,2590.0,26194.0,0.08,1.23,12.30,13.68,37.0,0.10,1.53,17.96,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-22.52,40.0,H-LC,4.01,206322.0,1822.0,39593.0,1.21,0.89,19.19,20.25,10.0,0.05,1.48,9.70,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,30.0,H-LC,12.41,272732.0,-73224.0,148093.0,0.45,-21.17,54.30,21.64,1.0,-0.49,1.96,0.45,X40,ATH,IT
83,VBL,671.64,-21.40,32.0,H-LC,9.26,282889.0,-32953.0,145603.0,-0.07,-10.43,51.47,35.67,5.0,-0.23,2.03,1.98,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.95,32.0,H-LC,10.26,217272.0,-24652.0,87930.0,-0.09,-10.19,40.47,26.15,8.0,-0.28,1.56,2.67,X40,ATH,IT
40,INFY,2275.00,-21.75,37.0,H-LC,8.21,305463.0,-7425.0,179124.0,-0.53,-2.37,58.64,54.87,3.0,-0.04,2.19,4.12,X40,BTT,IT
42,ITC,452.00,-38.77,47.0,H-LC,2.04,198021.0,-2117.0,23010.0,0.85,-1.06,11.62,10.44,4.0,-0.09,1.42,4.45,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-3.25,43.0,H-LC,1.98,155605.0,-30490.0,77974.0,-0.55,-16.38,50.11,25.51,15.0,-0.39,1.12,14.67,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,47.0,H-LC,2.04,198021.0,-2117.0,23010.0,0.85,-1.06,11.62,10.44,4.0,-0.09,1.42,4.45,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-18.81,27.0,H-LC,5.82,204776.0,-46992.0,100197.0,-0.98,-18.66,48.93,21.13,27.0,-0.47,1.47,10.15,X40,ATH,PAINTS
19,CIPLA,1795.00,-22.52,40.0,H-LC,4.01,206322.0,1822.0,39593.0,1.21,0.89,19.19,20.25,10.0,0.05,1.48,9.70,X40N,BTT,PHARMA
57,RELIANCE,1533.00,-14.79,40.0,H-LC,3.70,212956.0,2590.0,26194.0,0.08,1.23,12.30,13.68,37.0,0.10,1.53,17.96,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,30.0,H-LC,12.41,272732.0,-73224.0,148093.0,0.45,-21.17,54.30,21.64,1.0,-0.49,1.96,0.45,X40,ATH,IT
40,INFY,2275.00,-21.75,37.0,H-LC,8.21,305463.0,-7425.0,179124.0,-0.53,-2.37,58.64,54.87,3.0,-0.04,2.19,4.12,X40,BTT,IT
42,ITC,452.00,-38.77,47.0,H-LC,2.04,198021.0,-2117.0,23010.0,0.85,-1.06,11.62,10.44,4.0,-0.09,1.42,4.45,X40,NTT,FMCG
83,VBL,671.64,-21.40,32.0,H-LC,9.26,282889.0,-32953.0,145603.0,-0.07,-10.43,51.47,35.67,5.0,-0.23,2.03,1.98,X40N,ATH,FMCG
1,ABB,7934.00,-40.45,44.0,H-LC,7.70,248880.0,-12739.0,131956.0,0.22,-4.87,53.02,45.57,7.0,-0.10,1.79,5.90,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,46.0,L-SC,40.40,80352.0,-33197.0,73249.0,3.51,-29.24,91.16,35.27,268.0,-0.45,0.58,96.66,XR,NTT,HOTELS
49,MASFIN,398.61,-20.96,34.0,H-SC,12.27,89655.0,-8325.0,29927.0,-1.26,-8.50,33.38,22.05,152.0,-0.28,0.64,30.32,XR,ATH,FINANCE
50,NATIONALUM,244.55,-43.66,62.0,H-MC,12.52,113070.0,12566.0,16542.0,-0.25,12.50,14.63,28.96,79.0,0.76,0.81,50.72,MH,ATH,METALS
78,UJJIVANSFB,60.00,105.37,64.0,H-SC,11.45,129735.0,-12744.0,32265.0,3.67,-8.94,24.87,13.70,163.0,-0.39,0.93,53.12,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-186.36,80.0,M-SC,8.28,149931.0,711.0,143469.0,3.56,0.48,95.69,96.62,208.0,0.00,1.08,62.26,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1244.50,69.0,M-SC,14.28,182779.0,1636.0,43830.0,6.73,0.90,23.98,25.10,235.0,0.04,1.31,46.48,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.00,-186.36,80.0,M-SC,8.28,149931.0,711.0,143469.0,3.56,0.48,95.69,96.62,208.0,0.00,1.08,62.26,XY25,NTT,FINANCE
69,SURYODAY,240.00,68.86,78.0,H-SC,1.76,155907.0,-23164.0,88898.0,15.02,-12.94,57.02,36.71,135.0,-0.26,1.12,54.69,XR,NTT,BANKS
50,NATIONALUM,244.55,-43.66,62.0,H-MC,12.52,113070.0,12566.0,16542.0,-0.25,12.50,14.63,28.96,79.0,0.76,0.81,50.72,MH,ATH,METALS
26,GREENPANEL,537.00,235.27,59.0,M-SC,1.39,149743.0,-33335.0,112307.0,11.18,-18.21,75.00,43.14,230.0,-0.30,1.08,40.85,XY24,NTT,MISC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.02
1,25,43.07
2,50,74.87


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.28
LC    32.08
MC    23.69
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.95
X40      14.47
X40N     11.22
XR       11.10
XY25     10.47
AR        8.12
OX40N     7.79
X200      1.65
MH        1.61
X5K       1.50
SR        1.17
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.00
H-LC    25.47
H-MC    20.40
M-SC    13.83
M-LC     5.57
M-MC     2.99
L-SC     1.45
L-LC     1.04
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.83,-6.57,41.74
IT,12.47,-21.14,87.19
FINANCE,10.37,-16.87,69.95
MISC,7.01,-18.02,83.63
BANKS,6.44,-11.32,69.89
PAINTS,5.52,-19.98,37.86
ELECTRICAL,5.36,-10.60,49.41
INSURANCE,3.76,-5.81,45.37
AUTO,3.56,-16.10,71.60


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3266488.0,22
XR,1366885.0,14
AR,1203297.0,9
X40,835066.0,10
X40N,642819.0,9
OX40N,562269.0,10
XY25,481042.0,7
SR,259490.0,2
X5K,108203.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3759641.0,29
M-SC,1507044.0,17
H-LC,1274303.0,15
H-MC,1267337.0,15
M-LC,375238.0,4
M-MC,367146.0,2
L-SC,260744.0,3
L-MC,60379.0,1
L-LC,39795.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1238930.0      6
M-SC       XY24       846259.0      7
H-SC       AR         823719.0      5
           XR         791556.0      7
H-MC       XY24       581951.0      4
H-LC       X40        538354.0      5
M-MC       XY24       367146.0      2
H-SC       X40N       315639.0      4
           SR         259490.0      2
           OX40N      257146.0      4
H-LC       X40N       236889.0      3
H-MC       X40        221335.0      4
H-LC       AR         209930.0      2
H-MC       XY25       181973.0      2
L-SC       XR         171872.0      2
M-SC       AR         169648.0      2
M-LC       XY24       155242.0      2
M-SC       XY25       143469.0      1
           OX40N      141329.0      4
           XR         130962.0      2
H-LC       XY25       115805.0      3
M-LC       XR         111793.0      1
           X5K        108203.0      1
H-MC       XR         100323.0      1
H-LC       X200        96365.0      1
H-MC       X40N        90291.0      2
L-SC       OX40N       88872.0      1
H-LC       XY24        76960.0      1
M-SC       X40         75377.0      1
H-MC       OX40N       74922.0      1
H-SC       MH          73161.0      1
L-MC       XR          60379.0      1
L-LC       XY25        39795.0      1
H-MC       MH          16542.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 35.0 seconds
